```
pip install snowflake-connector-python[secure-local-storage,pandas]
pip install sqlalchemy
```

In [ ]:
import re
import pandas
import getpass
import json

from tabulate import tabulate

import sqlalchemy.exc
from sqlalchemy import create_engine

from sqlalchemy.dialects import registry

registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

SAML_USERNAME = f'{getpass.getuser()}@volvocars.com'  # or SAML_USERNAME = 'cds-id@volvocars.com'
engine = create_engine('snowflake://volvocars-manufacturinganalytics' ,connect_args={'user': SAML_USERNAME,'authenticator': 'externalbrowser',})

reduced_dataset = False
query_rowLimit = "" if not reduced_dataset else " top 1000"

### notebook helper functions

In [ ]:
def query_snowflake(query:str, engine=engine) -> pandas.DataFrame:
    """Query snowflake and return a pandas dataframe"""
    connection = engine.connect()
    try:
        _resultSet = pandas.read_sql_query(query, connection)
    except (sqlalchemy.exc.ProgrammingError, sqlalchemy.exc.OperationalError) as e:
        print(f'Error: {e}')
        _resultSet = pandas.DataFrame()
    finally:
        connection.close()
        engine.dispose()
    return _resultSet

def pivot_and_resample(df: pandas.DataFrame, column_name: str, interval: str) -> pandas.DataFrame:
    """Pivot and resample a dataframe"""
    df = df.reset_index().pivot_table(index='timestamp', columns=column_name, values='value')
    df = df.resample(interval).median()
    df = df.round(3)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

def tabulate_table(df: pandas.DataFrame, rows:int = 15) -> None:
    """Tabulate a dataframe"""
    print(tabulate(df.head(rows).astype(str), headers='keys', tablefmt='pipe', showindex='always', ))

class process:
    def __init__(self) -> None:
        pass

    def detect_pattern(self, input: str):
        pattern = re.compile(
            r"^(?:_|B)?"
            + r"(?P<abbr1>[A-Z]{2}\B)"  # VP
            + r"(?P<fake_location>(?P<level>\d)..[PM]\d(?:\d)?)"  # 413P08
            + r"(?P<abbr2>[A-Z]*)?(?P<numb2>[0-9]*)?"  # M1
            + r"(?P<abbr3>[A-Z]*)?(?P<numb3>[0-9]*)?"  # UE1
            + r"(?P<abbr4>[A-Z]*)?(?P<numb4>[0-9]*)?"
        )
        match = re.match(pattern, input)
        return match.groupdict() if match else {}

### query tracking data
todo


### query machine metadata

In [ ]:
# query definitions from machines
query = """
    select distinct "a_area", "a_process", "a_zone", "object", "a_machinedesc", "a_componentdesc"
    from VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
    where date_trunc('Y',"timestamp" ) > '2022'
    and "a_process" like '%oven%'
    and "a_machinedesc" is not null
    and "a_machinedesc" != ''
    and "a_componentdesc" is not null
    and "a_componentdesc" != ''
    order by "a_area", "a_process", "a_zone"
    """
definitions_machines = query_snowflake(query)

In [ ]:
definitions_machines

In [ ]:
# for row in definitions_machines, check against process.detect_pattern(row['object'])
# if match add "fake_location" to row['zone_code']
for index, row in definitions_machines.iterrows():
    process_object = process()
    pattern = process_object.detect_pattern(row['object'])
    if pattern:
        definitions_machines.loc[index, 'zone_code'] = pattern['fake_location'][1:]
    else:
        definitions_machines.loc[index, 'zone_code'] = ""
tabulate_table(definitions_machines)


In [ ]:
# create new dataframe, with index zone_code, and join all object that have the same zone_code
machine_map = definitions_machines[['zone_code','object', 'a_zone']].copy().drop_duplicates()
# order by zone_code
machine_map.sort_values(by=['zone_code'], inplace=False)
tabulate_table(machine_map, rows=100)

### query plc function block metadata

In [ ]:
# query definitions from function block types and variables
query = """
    select distinct "property", "property_desc", "fb_type", "fb_description"
    from VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
    where date_trunc('Y',"timestamp" ) > '2022'
    and "a_process" like '%oven%'
    and "property_desc" is not null
    and "property_desc" != ''
    """
definitions_variables = query_snowflake(query)

In [ ]:
tabulate_table(definitions_variables)

### variables for query's

In [ ]:
# create a list of column names
wanted_columns = ['timestamp','object','plc','value','unit','property','value_condition']
timestamp_columns = ["""date_part('Y',"timestamp" ) year""", """date_part('WEEKISO',"timestamp" ) week""", """date_part('DAYOFWEEKISO',"timestamp" ) day""" ]

In [ ]:
wanted_ovens = ['electrocoat oven 1','electrocoat oven 2']

### resultset actual temperatures
joined with process code

In [ ]:
# query temperature readings
query = f"""
    SELECT {query_rowLimit} "{'","'.join(wanted_columns)}", {','.join(timestamp_columns)}
    FROM VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
    WHERE "a_process" = 'electrocoat oven 1' 
    AND "fb_type" = 'FB1700_LAnInput6Lim'
    AND "a_componentdesc" = 'Temperature sensor'
    and year = '2022'

    """

In [ ]:
resultSet = query_snowflake(query)
resultSet = resultSet.merge(machine_map[['object','zone_code']], on='object', how='left')


### resultset for setpoints
joined with process code

In [ ]:
# query setpoints
query = f"""
    SELECT "{'","'.join(wanted_columns)}", {','.join(timestamp_columns)} 
    FROM VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
    WHERE "a_process" = 'electrocoat oven 1'
    AND "property" = 'H_SP1'
    AND "unit" = '°C'
    and year = '2022'

    """
resultSet_setpoints = query_snowflake(query)  # resultSet_setpoints.dtypes
resultSet_setpoints = resultSet_setpoints.merge(machine_map[['object','zone_code']], on='object', how='left')


### start stop times of oven

In [ ]:
# query setpoint index changes (op and close time)
query = f"""
    -- open times
    select min("timestamp") timestamp, year, week, day, 'start-request' as type
    from (
        select "timestamp","object","plc","value",
            lag("value") over ( partition by "object" order by "timestamp") as previous_value,
            "unit","property", "property_desc",
            date_part('Y',"timestamp" ) year,
            date_part('WEEKISO',"timestamp" ) week,
            date_part('DAYOFWEEKISO',"timestamp" ) day
            
        from VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
        where "a_process" = 'electrocoat oven 1'
        --and "object" = 'ER6E1P10VM1'
        and "property" = 'I_SPReq'
        and "property_desc" is not null
        and "property_desc" != ''
        and year = '2022'

    )
    where "value" <> previous_value
    and previous_value = 0
    group by year, week, day
    union all
    -- closing times
    select max("timestamp") timestamp, year, week, day, 'stop-request' as type
    from (
        select "timestamp","object","plc","value",
            lead("value") over ( partition by "object" order by "timestamp") as next_value,
            "unit","property", "property_desc",
            date_part('Y',"timestamp" ) year,
            date_part('WEEKISO',"timestamp" ) week,
            date_part('DAYOFWEEKISO',"timestamp" ) day
            
        from VCG.GB_PLC_300_TELEMETRICS_V1.FROMHMI
        where "a_process" = 'electrocoat oven 1'
        --and "object" = 'ER6E1P10VM1'
        and "property" = 'I_SPReq'
        and "property_desc" is not null
        and "property_desc" != ''
        and year = '2022'
        
    )
    where "value" <> next_value
    and next_value = 0
    group by year, week, day
"""

resultSet_start_stop = query_snowflake(query)  # resultSet_start_stop.dtypes
resultSet_start_stop['date'] = pandas.to_datetime(resultSet_start_stop['timestamp']).dt.date # add date column


In [ ]:
# create table with start and stop times
start_stop_times = resultSet_start_stop.reset_index().pivot(index='date',  columns ='type', values='timestamp')
# backfill stop-request
start_stop_times['stop-request'] = start_stop_times['stop-request'].fillna(method='bfill')
start_stop_times = start_stop_times[start_stop_times['start-request'].notnull()]

#remove multi-index
start_stop_times = start_stop_times.reset_index()
start_stop_times = start_stop_times.set_index('date')

# convert to datetime, round to nearest minute
start_stop_times['start-request'] = start_stop_times['start-request'].dt.round('min')
start_stop_times['stop-request'] = start_stop_times['stop-request'].dt.round('min')

tabulate_table(start_stop_times)

## identify heating sequence
1) use start-sequence  -> all zones have temp ok  ->  (5days production) ->  stop-request (X5 bigger)
2) use start-sequence  -> all zones have temp ok  -> stop-request


production ok = all zones reached setpoint_ok

In [ ]:
resultSet

### production ok temperature

In [ ]:

condition_words = ['Productie','Productie OK','Production OK','Produktie','Produktie OK', 'Start OK']
setpoint_ok = pivot_and_resample(resultSet[resultSet['value_condition'].isin(condition_words) & (resultSet['object'] != 'KP6E1P91BT1') | (resultSet['object'] == 'KP6E1P06BT1')], 'zone_code', '12H')
setpoint_ok['condition'] = 'setpoint_ok'
#tabulate_table(setpoint_ok, rows=15)
setpoint_ok
setpoint_ok.reset_index()

In [ ]:
setpoint_ok.iloc[:,1]

### to low temperature

In [ ]:
condition_words = ['Laag limiet','Laag','Laag Limiet','Minimum','Te Laag', 'Te laag']
setpoint_low = pivot_and_resample(resultSet[resultSet['value_condition'].isin(condition_words)], 'zone_code', '12H' )
setpoint_low['condition'] = 'setpoint_low'
setpoint_low = setpoint_low[setpoint_low.columns.intersection(setpoint_ok.columns)] # remove columns not in setpoint_ok
tabulate_table(setpoint_low, rows=15)

### to high temperature

In [ ]:
condition_words = ['Hoog limiet','Hoog','Hoog Limiet','Maximum','Te Hoog', 'Te hoog']
setpoint_high = pivot_and_resample(resultSet[resultSet['value_condition'].isin(condition_words)], 'zone_code', '12H' )
setpoint_high['condition'] = 'setpoint_high'
setpoint_high = setpoint_high[setpoint_high.columns.intersection(setpoint_ok.columns)] # remove columns not in setpoint_ok
tabulate_table(setpoint_high, rows=15)

### setpoint temperature

In [ ]:
setpoints = pivot_and_resample(resultSet_setpoints, 'zone_code', '12H' )
setpoints = setpoints[setpoints.columns.intersection(setpoint_ok.columns)] # remove columns not in setpoint_ok
setpoints['condition'] = 'setpoint'
tabulate_table(setpoints, rows=50)

In [ ]:
# concat all setpoints
conditions = pandas.concat([setpoint_ok, setpoint_low, setpoint_high, setpoints])
#aggregate all setpoints by condition, drop timestamp
conditions = conditions.groupby('condition').median()

tabulate_table(conditions, rows=50)

### actual values

In [ ]:
# create subset where property is O_AnInputScaled
values = pivot_and_resample(resultSet[resultSet['property'] == 'O_AnInputScaled'], 'zone_code', '1min')
values = values[values.columns.intersection(setpoint_ok.columns)] # remove columns not in setpoint_ok

tabulate_table(values, rows=15)

In [ ]:
setpoint_low

In [ ]:
values

In [ ]:
#!pip install plotly-resampler

In [ ]:
for col in values.columns:
    print(col)

In [ ]:
values.columns[0]

In [ ]:
import plotly.graph_objs as go
import plotly_resampler as pr

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=values.index, y=values['E1P20'], mode='lines', name='E1P20'))
fig.add_trace(go.Scatter(x=values.index, y=values['E1P10'], mode='lines', name='E1P10'))
fig.add_trace(go.Scatter(x=values.index, y=values['E1P30'], mode='lines', name='E1P30'))
fig.add_trace(go.Scatter(x=values.index, y=values['E1P80'], mode='lines', name='E1P80'))
fig.show()


In [ ]:
fig = go.Figure()
for col in values.columns:
    fig.add_trace(go.Scatter(x=values.index, y=values[col], mode='lines', name=col))
fig.show()

In [ ]:
#since the temperature of the oven is very uniform the last few hours before starting up, we can use this to check when it suddenly sharply rises and we can tell that the oven was turned on.
#If temp rose more than x degrees higher than lowest value of last half hour: oven on

In [ ]:
min_E1P10 = values['E1P10'].rolling('30min').min()
min_E1P10

In [ ]:
import pandas as pd

In [ ]:
# Read the table into a pandas DataFrame

'''
We're only interested in the heating curve.

Check the temperature of the last 30 minutes. Right before the oven gets turned on, the temperature is stable and stays about the same value. 
If the temperature rapidly rises above 6 degrees C compared to the lowest value of the last 30 mins, we can conclude
that the oven has been turned on and we tag these values as "on". We continue to tag the temp values as "on" until we reach the 
minimum setpoint temperature.

'''




list = []

for col in values.columns:
    df = values.copy()
    
    #only consider tagging values below this temp value
    value = conditions.loc['setpoint_low', col]

# Create a new column called "lowest_E1P10" and set its value to the lowest value of the "E1P10" column in the last 30 minutes for each row
    df['lowest_'+col] = df[col].rolling('30min', min_periods=1).min()

# Create a new column called "difference_E1P10" and set its value to the difference between the value in the "E1P10" column and the value in the "lowest_E1P10" column for each row
    df['difference_'+col] = df[col] - df['lowest_'+col]


#if temp is lower than the lowest acceptable setpoint temp (aka setpoint_low), and diff is greater than 3, then oven is on
    df.loc[(df[col] < value) & (df['difference_'+col] > 6), col+'on_off'] = 'on'
    df.loc[~((df[col] < value) & (df['difference_'+col] > 6)), col+'on_off'] = 'off'
    df_col = df
    del df
    list.append(df_col)


In [ ]:
df2 = list[1].loc['2022-08-01 23:59:00':'2022-12-31 23:59:00']

In [ ]:
import plotly.express as px

col = values.columns[1]
# Plot E1P10 values in black
fig = px.scatter(df2, x=df2.index, y='E1P10', color_discrete_sequence=['white'])

# Plot "on" values of E1P10 in red
fig.add_scatter(x=df2.index[df2[col+'on_off'] == 'on'], y=df2['E1P10'][df2[col+'on_off'] == 'on'], mode='markers', marker=dict(color='black'))

# Set the title and axis labels
fig.update_layout(title='E1P10 temp values', xaxis_title='Timestamp', yaxis_title='E1P10 temp (C)')

# Show the plot
fig.show()

In [ ]:
list[1]

In [ ]:
list2 = []
for df in list:
    df3 = df[df.iloc[:, -1] == 'on']
    list2.append(df3)


In [ ]:
list2[1]

In [ ]:
list3 = []
for df3 in list2:
    on_stretches = []
    current_stretch = []
    for index, row in df3.iterrows():
        if not current_stretch:
            current_stretch.append(index)
        else:
            time_diff = index - current_stretch[-1]
            if time_diff > pd.Timedelta(minutes=1):
                on_stretches.append(current_stretch)
                current_stretch = [index]
            else:
                current_stretch.append(index)
    if current_stretch:
        on_stretches.append(current_stretch)
    list3.append(on_stretches)


In [ ]:
list3[0]

In [ ]:
len(list3[0])

In [ ]:
list4=[]
for timestamps in list3:
    
    oven_warmup_times = []
    for list in timestamps:
        oven_warmup_times.append(len(list))
    list4.append(oven_warmup_times)

In [ ]:
list4[1]

In [ ]:
list5 = []
for warmup_times in list4:
    list6 = []
    list6 = [x for x in warmup_times if x >= 5]
    list5.append(list6)


In [ ]:
list5[1]

In [ ]:
print(len(list5))

In [ ]:
average = sum(oven_warmup_times) / len(oven_warmup_times)

In [ ]:

conditions_with_oven_temp = conditions.copy()
new_row = {'E1P10': average, 'E1P06': 'z', 'E1P20': 'z', 'E1P30': 'x', 'E1P40': 'z', 'E1P50': 'z', 'E1P60': 'x', 'E1P70': 'z', 'E1P80': 'z'}
conditions_with_oven_temp.loc[len(conditions_with_oven_temp)] = new_row

In [ ]:
conditions_with_oven_temp = conditions_with_oven_temp.rename(index={4: 'avg_oven_warmup_time(minutes)'})

In [ ]:
conditions

In [ ]:
conditions_with_oven_temp

In [ ]:
conditions

In [ ]:
list7 = []
for zone in list3:
    list8 = []
    list8 = [x for x in zone if len(x) >= 5]
    list7.append(list8)

In [ ]:



max_warmup_times = pd.DataFrame()
for i in range(1,(len(list5))):

    max_warmup_times[values.columns[i]] = [max(list5[i])]
max_warmup_times.rename(index={0: 'max warmup time (minutes)'}, inplace=True)
max_warmup_times

In [ ]:
value

In [ ]:
del times
times = pd.DataFrame(columns=['Begin time', 'End time', 'Time difference', 'Begin temp (C)', 'End temp (C)', 'Temperature diff (C)'])
times.name = values.columns[1]
for interval in list7[1]:
    begin_time = interval[0]
    end_time = interval[-1]
    time_delta = str(end_time - begin_time)[7:]
    begin_temp = values.loc[interval[0],times.name]
    end_temp = values.loc[interval[-1],times.name]
    temp_delta = end_temp - begin_temp
    times.loc[len(times)] = [begin_time, end_time, time_delta, begin_temp ,end_temp, temp_delta]

In [ ]:
print("detected warmup curves for zone:" + times.name + "\n")
times

### create excel

In [ ]:
excel_file = 'oven_data.xlsx'
page_1 = 'measurements'
page_2 = 'conditions'
page_3 = 'start_stop_times'

with pandas.ExcelWriter(excel_file) as writer:
    values.to_excel(writer, sheet_name=page_1)
    conditions.to_excel(writer, sheet_name=page_2)
    start_stop_times.to_excel(writer, sheet_name=page_3)



note: *I excelled myself by making this notebook*